In [40]:
query='tel me 5 news about quantum computing and last 5 about cloud computing? If you dont know how to answer'

In [41]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
import logging
import sys
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
import logging
sys.path.append(r"C:\Users\ELAFACRB1\Codice\GitHub\media-chat-service\src\embedding")
from utils import ChromaDBManager, LangChainAI, QDrantDBManager, EmbeddingFunction
from langchain.chains import RetrievalQA

In [42]:
embedding = EmbeddingFunction('openAI').embedder
vectore_store=qdrantClient = QDrantDBManager(
    url="http://ec2-18-209-145-26.compute-1.amazonaws.com:6333/dashboard",
    port=6333,
    collection_name="rio-rag-platform",
    vector_size=1536,
    embedding=embedding,
    record_manager_url=r"sqlite:///C:\Users\ELAFACRB1\Codice\GitHub\media-chat-service\src\embedding\record_manager_cache.sql"
)
vectore_store_client=vectore_store.vector_store

Collection rio-rag-platform already exists!


In [43]:
llm = ChatOpenAI(temperature=0)

In [44]:
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [45]:
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectore_store_client.as_retriever(), 
    llm=llm
)

In [46]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=retriever_from_llm,
    return_source_documents=True,
)

In [47]:
res=qa_chain(query)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the concept of quantum computing?', '2. Can you provide information on the principles behind quantum computing?', '3. How does quantum computing differ from classical computing?']


In [48]:
res['result']

'Quantum computing operates using qubits that can be both 0 and 1 simultaneously, unlike classical computers that use bits. This unique feature allows quantum computers to process massive amounts of information all at once, giving them incredible power. By encoding information into quantum systems like atoms or photons, quantum computers apply the laws of quantum mechanics to solve problems with enhanced speed, efficiency, and insight.'

In [38]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [39]:
response_schemas = [
    ResponseSchema(
        name="title", 
        description="title of the news"
    ),
    ResponseSchema(
        name="text",
        description="long text (at least 3 row long) of the retrieved news",
    ),
    ResponseSchema(
        name="url",
        description="url of the news"
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions=output_parser.get_format_instructions()